In [36]:
from twisted.internet import asyncioreactor
asyncioreactor.install()

In [2]:
import os

In [7]:
import httpx

In [22]:
import attr

In [28]:
import ipywidgets

In [38]:
import dateutil

In [45]:
from dateutil import parser, tz

In [42]:
import datetime

In [53]:
from twisted.internet import reactor, task

In [60]:
import functools

In [23]:
@attr.s(auto_attribs=True, frozen=True)
class Project:
    name: str
    pid: int
        
    @classmethod
    def from_api(cls, entry):
        return cls(name=entry["name"], pid=entry["id"])

In [5]:
with open(os.path.expanduser("~/.toggl-token")) as fpin:
    token = fpin.read().strip()

In [8]:
client = httpx.Client(auth=(token, "api_token"))

In [51]:
def get_current(client, now):
    res = client.get("https://api.track.toggl.com/api/v8/time_entries/current")
    data = res.json()["data"]
    description = data["description"]
    since = parser.isoparse(data["start"])
    duration = now - since
    minutes, seconds = divmod(duration, datetime.timedelta(minutes=1))
    seconds = int(seconds.total_seconds())
    return description, minutes, seconds

In [16]:
res = client.get("https://api.track.toggl.com/api/v8/workspaces")
[workspace] = res.json()
workspace_id = workspace["id"]

In [18]:
res = client.get(f"https://api.track.toggl.com/api/v8/workspaces/{workspace_id}/projects")

In [24]:
projects = list(map(Project.from_api, res.json()))

In [52]:
current = ipywidgets.Label("")
current

Label(value='')

In [64]:
def update(label, client, get_now):
    description, minutes, seconds = get_current(client, get_now())
    label.value = f"{description} {minutes}:{seconds:02d}"

In [64]:
update(current, client, functools.partial(datetime.datetime.now, tz.UTC))

In [30]:
project_selection=ipywidgets.Combobox(
    options=[project.name for project in projects],
    description='Project',
    ensure_option=True,
    disabled=False
)
description = 

Combobox(value='', description='Project', ensure_option=True, options=('Entertainment', 'Health', 'Household',…